In [1]:
import pandas as pd
import numpy as np

### Ticket sales in Europe

In [2]:
df_admissions_EU = pd.read_csv('./data/Movie_numbers_1996-2023_EU.csv')

In [ ]:
df_admissions_EU.isnull().sum()

In [ ]:
df_admissions_EU.info()

In [ ]:
df_admissions_EU.head(20)

In [7]:
df_admissions_EU.rename(columns={'Admissions 1996-1996' : 'tickets_sold',
                                 'Total EU27+GB since 1996' : 'tickets_sold_since_1996',
                                 'Original title' : 'title',
                                 'Production year' : 'year'}, inplace=True)
df_admissions_EU.columns = df_admissions_EU.columns.str.lower().str.replace(' ', '_')
df_admissions_EU.drop(columns=['directors'], inplace=True)

In [8]:
df_admissions_EU['tickets_sold'] = df_admissions_EU['tickets_sold'].str.replace(' ', '')
df_admissions_EU['tickets_sold_since_1996'] = df_admissions_EU['tickets_sold_since_1996'].str.replace(' ', '')

In [9]:
df_admissions_EU['tickets_sold'] = pd.to_numeric(df_admissions_EU['tickets_sold'])
df_admissions_EU['tickets_sold_since_1996'] = pd.to_numeric(df_admissions_EU['tickets_sold_since_1996'])

In [ ]:
df_admissions_EU.shape

In [ ]:
df_admissions_EU['title'].nunique()

In [ ]:
df_admissions_EU_clean = df_admissions_EU.groupby(['title', 'producing_country', 'year', 'tickets_sold_since_1996']).sum().reset_index()
df_admissions_EU_clean

In [ ]:
display(df_admissions_EU_clean[df_admissions_EU_clean['title'] == 'Avatar'])
display(df_admissions_EU[df_admissions_EU['title'] == 'Avatar'])

In [ ]:
df_admissions_EU_clean.head(20)

### Ticket sales in North America


In [ ]:
df_admissions_NA = pd.read_csv('./Data/Movie_numbers_1996-2023_USA.csv')
df_admissions_NA.head(20)

In [ ]:
df_admissions_NA.info()

In [17]:
df_admissions_NA['Release\nDate'] = pd.to_datetime(df_admissions_NA['Release\nDate'])
df_admissions_NA['Tickets Sold'] = df_admissions_NA['Tickets Sold'].str.replace(',', '')
df_admissions_NA['Tickets Sold'] = pd.to_numeric(df_admissions_NA['Tickets Sold'])

In [ ]:
df_admissions_NA['1996 Gross'] = df_admissions_NA['1996 Gross'].str.replace(',', '')
df_admissions_NA['1996 Gross'] = df_admissions_NA['1996 Gross'].str.replace('$', '')

In [19]:
df_admissions_NA['1996 Gross'] = pd.to_numeric(df_admissions_NA['1996 Gross'])

In [ ]:
df_admissions_NA.rename(columns={'Release\nDate' : 'release_date',
                                 'Movie' : 'title',
                                 '1996 Gross' : 'Gross_Sales',
                                 'Tickets Sold' : 'tickets_sold'}, inplace=True)
df_admissions_NA.columns = df_admissions_NA.columns.str.lower()
df_admissions_NA.columns

In [ ]:
df_admissions_NA['title'].nunique()

In [ ]:
df_admissions_NA_clean = df_admissions_NA.groupby(['title', 'release_date', 'distributor']).sum().reset_index()
df_admissions_NA_clean.drop(columns=['rank', 'year'], inplace=True)
df_admissions_NA_clean.shape

In [ ]:
display(df_admissions_NA_clean[df_admissions_NA_clean['title'] == 'Avatar'])
display(df_admissions_NA[df_admissions_NA['title'] == 'Avatar'])

In [ ]:
df_admissions_NA_clean.groupby(['title']).size().sort_values(ascending=False)

In [ ]:
df_admissions_NA_clean[df_admissions_NA_clean['title'] == 'Halloween']

In [ ]:
display(df_admissions_NA_clean.head())
display(df_admissions_EU_clean.head())

### IMDB DATA

In [27]:
df_movie_list_years = pd.read_csv('./Data/movie_list_years.csv')
df_akas = pd.read_csv('./Data/title.akas.csv')
df_basics = pd.read_csv('./Data/title.basics.csv')
df_crew = pd.read_csv('./Data/title.crew.csv')
df_principals = pd.read_csv('./Data/title.principals.csv')
df_ratings = pd.read_csv('./Data/title.ratings.csv')
df_names = pd.read_csv('./Data/name.basics.csv')

In [28]:
df_movie_list_years.replace('\\N', np.nan, inplace=True)
df_akas.replace('\\N', np.nan, inplace=True)
df_basics.replace('\\N', np.nan, inplace=True)
df_crew.replace('\\N', np.nan, inplace=True)
df_principals.replace('\\N', np.nan, inplace=True)
df_ratings.replace('\\N', np.nan, inplace=True)
df_names.replace('\\N', np.nan, inplace=True)

In [ ]:
display(df_ratings.head(1))

In [30]:
df_crew_named = df_crew.merge(df_names, how='left', left_on='directors', right_on='nconst')
df_crew_named.drop(columns=['nconst', 'birthYear', 'deathYear', 'primaryProfession'], inplace=True)

In [31]:
df_crew_named.rename(columns={'primaryName' : 'directors_name',
                              'knownForTitles' : 'known_for_directing'}, inplace=True)

In [ ]:
df_crew_named['writers'].str.count(',').sort_values(ascending=False)

In [ ]:
df_crew_named.iloc[427635]

### IMDB Final / Merging

In [34]:
df_imdb_final = df_movie_list_years.merge(df_crew, how='left', on='tconst')

In [ ]:
df_imdb_final.shape

In [ ]:
df_imdb_final['directors'].str.count(',').nunique()

In [ ]:
df_imdb_final['directors_count'] = df_imdb_final['directors'].str.count(',') + 1
df_imdb_final['directors_count'].value_counts()/len(df_imdb_final) * 100

In [38]:
df_imdb_final[['director', 'director2', 'director3']] = df_imdb_final['directors'].str.split(',', n=2, expand=True)
df_imdb_final.drop(columns='directors', inplace=True)

In [ ]:
df_imdb_final['writers_count'] = df_imdb_final['writers'].str.count(',') + 1
df_imdb_final['writers_count'].value_counts()/len(df_imdb_final[df_imdb_final['writers_count'].notnull()]) * 100

In [40]:
df_imdb_final[['writer', 'writer2', 'writer3']] = df_imdb_final['writers'].str.split(',', n=2, expand=True)
df_imdb_final.drop(columns='writers', inplace=True)

In [ ]:
df_imdb_final

### Set option

In [123]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [43]:
df_imdb_final = df_imdb_final.merge(df_names[['nconst', 'primaryName']], how='left', left_on='director', right_on='nconst')
df_imdb_final.drop(columns='nconst', inplace=True)
df_imdb_final.rename(columns={'primaryName' : 'director_name'}, inplace=True)
df_imdb_final = df_imdb_final.merge(df_names[['nconst', 'primaryName']], how='left', left_on='director2', right_on='nconst')
df_imdb_final.drop(columns='nconst', inplace=True)
df_imdb_final.rename(columns={'primaryName' : 'director2_name'}, inplace=True)
df_imdb_final = df_imdb_final.merge(df_names[['nconst', 'primaryName']], how='left', left_on='director3', right_on='nconst')
df_imdb_final.drop(columns='nconst', inplace=True)
df_imdb_final.rename(columns={'primaryName' : 'director3_name'}, inplace=True)

In [44]:
df_imdb_final = df_imdb_final.merge(df_names[['nconst', 'primaryName']], how='left', left_on='writer', right_on='nconst')
df_imdb_final.drop(columns='nconst', inplace=True)
df_imdb_final.rename(columns={'primaryName' : 'writer_name'}, inplace=True)
df_imdb_final = df_imdb_final.merge(df_names[['nconst', 'primaryName']], how='left', left_on='writer2', right_on='nconst')
df_imdb_final.drop(columns='nconst', inplace=True)
df_imdb_final.rename(columns={'primaryName' : 'writer2_name'}, inplace=True)
df_imdb_final = df_imdb_final.merge(df_names[['nconst', 'primaryName']], how='left', left_on='writer3', right_on='nconst')
df_imdb_final.drop(columns='nconst', inplace=True)
df_imdb_final.rename(columns={'primaryName' : 'writer3_name'}, inplace=True)

In [ ]:
df_imdb_final

### Principals

In [ ]:
df_principals['category'].isin(['actor', 'actress'])

In [ ]:
df_principals['ordering'].unique()

In [ ]:
df_principals['tconst'].nunique()

In [49]:
# df_principals[df_principals['category'].isin(['actor', 'actress'])].groupby('ordering').size() / (df_principals['tconst'].nunique()) * 100

In [50]:
# df_principals[df_principals['category'].isin(['actor', 'actress'])].groupby('ordering').size() / (df_principals.loc[df_principals['category'].isin(['actor', 'actress']), ['tconst']].nunique()[0]) * 100

In [51]:
df_principals_filtered = df_principals.copy()

In [52]:
df_principals_filtered = df_principals_filtered[df_principals_filtered['category'].isin(['actor', 'actress'])]

In [ ]:
df_principals_filtered['category'].unique()

In [54]:
df_principals_filtered = df_principals_filtered[df_principals_filtered['ordering'] <= 5]

In [ ]:
df_principals_filtered['ordering'].unique()

In [ ]:
df_principals_filtered['tconst'].nunique()

In [57]:
df_principals_filtered = df_principals_filtered.merge(df_names[['nconst', 'primaryName']], how='left', on='nconst')

In [58]:
df_principals_filtered.rename(columns={'primaryName' : 'a_name'}, inplace=True)

In [59]:
df_principals_filtered = df_principals_filtered.pivot(index='tconst', columns='ordering', values='a_name')

In [60]:
df_principals_filtered = df_principals_filtered.reset_index()

In [ ]:
df_principals_filtered.rename_axis(None)

In [69]:
df_principals_filtered.rename_axis('', axis=1, inplace=True)

In [64]:
df_principals_filtered.rename(columns={1 : 'acting',
                                       2 : 'acting2',
                                       3 : 'acting3',
                                       4 : 'acting4',
                                       5 : 'acting5'}, inplace=True)

In [ ]:
df_principals_filtered

In [71]:
df_imdb_final = df_imdb_final.merge(df_principals_filtered, how='left', on='tconst')

In [72]:
df_akas.rename(columns={'titleId' : 'tconst'}, inplace=True)

### Ratings

In [73]:
df_imdb_final = df_imdb_final.merge(df_ratings, how='left', on='tconst')

In [ ]:
df_imdb_final

### Filtering

In [75]:
# Filtering for years
df_imdb_final = df_imdb_final[(df_imdb_final['startYear'] >= 1998) & (df_imdb_final['startYear'] <= 2023)]

In [ ]:
df_imdb_final[df_imdb_final['tconst'] == 'tt7846844']

In [77]:
df_imdb_final = df_imdb_final[df_imdb_final['titleType'] == 'movie']

In [78]:
df_imdb_final.drop(columns=['director', 'director2', 'director3', 'writer', 'writer2', 'writer3'], inplace=True)

In [79]:
df_imdb_final.rename(columns={'titleType' : 'title_type',
                              'primaryTitle' : 'primary_title',
                              'originalTitle' : 'original_title',
                              'isAdult' : 'is_adult',
                              'startYear' : 'year',
                              'endYear' : 'end_year',
                              'runtimeMinutes' : 'runtime',
                              'averageRating' : 'average_rating',
                              'numVotes' : 'num_votes'}, inplace=True)

In [80]:
df_imdb_final.drop(columns=['end_year'], inplace=True)

In [ ]:
df_imdb_final[df_imdb_final['average_rating'].notnull()].count()

In [ ]:
df_imdb_final.isnull().sum()

In [83]:
df_imdb_final.to_csv('./Data/df_imdb_final.csv')

In [ ]:
df_imdb_final.shape

### Genre cleaning

In [ ]:
df_imdb_final[df_imdb_final['genres'].str.count(',') == 2.0]

In [92]:
df_imdb_final['genres_count'] = df_imdb_final['genres'].str.count(',') + 1

In [ ]:
df_imdb_final.groupby('genres_count').size() / (len(df_imdb_final)) * 100

In [101]:
df_imdb_final[['genre', 'genre2', 'genre3']] = df_imdb_final['genres'].str.split(',', n=2, expand=True)

In [104]:
df_imdb_final.drop(columns='genres', inplace=True)

In [ ]:
df_imdb_final[df_imdb_final['genre'].isin(['Talk-Show', 'Reality-TV', 'Game-Show'])]

In [ ]:
df_imdb_final.head()

### Dropping NaN Ratings

In [ ]:
df_imdb_final[df_imdb_final['average_rating'].isnull()]

In [112]:
df_imdb_final = df_imdb_final[df_imdb_final['average_rating'].notnull()]

### WIEDER MIT GENRES BESCHAEFTIGEN

In [ ]:
df_imdb_final['genre'].unique()

In [166]:
df_imdb_final = df_imdb_final[~df_imdb_final['genre'].isin(['Talk-Show', 'Reality-TV'])]
df_imdb_final = df_imdb_final[~df_imdb_final['genre2'].isin(['Talk-Show', 'Reality-TV'])]
df_imdb_final = df_imdb_final[~df_imdb_final['genre3'].isin(['Talk-Show', 'Reality-TV', 'Game-Show'])]

In [ ]:
df_imdb_final[df_imdb_final['genre'] == 'News']

In [ ]:
df_imdb_final[df_imdb_final['genre2'] == 'News']

In [ ]:
df_imdb_final[df_imdb_final['genre3'] == 'News']

In [ ]:
df_imdb_final['genre2'].unique()

In [ ]:
df_imdb_final['genre3'].unique()

In [144]:
pd.options.mode.copy_on_write = True

In [145]:
df_imdb_final.replace({None : np.nan}, inplace=True)

In [ ]:
df_imdb_final[df_imdb_final['genre'].isnull()]

### Votes

In [ ]:
df_imdb_final[df_imdb_final['num_votes'] <= 5]

In [ ]:
df_imdb_final.columns

In [178]:
df_imdb_final.drop(columns='title_type', inplace=True)

In [181]:
df_imdb_final = df_imdb_final[['tconst', 'primary_title', 'original_title', 'year', 'runtime', 'num_votes', 'average_rating', 'genres_count', 'genre', 'genre2', 'genre3', 'acting', 'acting2', 'acting3', 'acting4', 'acting5', 
              'directors_count', 'director_name', 'director2_name', 'director3_name', 'writers_count', 'writer_name', 'writer2_name', 'writer3_name', 'is_adult']] 

In [ ]:
df_imdb_final

In [ ]:
display(df_admissions_EU_clean['title'].nunique())
display(df_admissions_NA_clean.shape)

In [ ]:
df_admissions_NA_clean['distributor'].nunique()

### Uploading to SQL

In [185]:
from sql_functions import get_engine, schema
import psycopg2
engine = get_engine()
schema = schema()
table_name = 'movie_data_EU'
table_name2 = 'movie_data_NA'
table_name3 = 'IMDB_data'

In [ ]:
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_admissions_EU_clean.to_sql(table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # your class schema
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

if engine!=None:
    try:
        df_admissions_NA_clean.to_sql(table_name2, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # your class schema
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name2} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

if engine!=None:
    try:
        df_imdb_final.to_sql(table_name3, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # your class schema
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name3} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None